# Requests

In [2]:
import requests

requests로 url이 어떤 HTTP method를 지원하는지 알려준다.

In [11]:
req_opt = requests.options('http://httpbin.org/')
type(req_opt)

requests.models.Response

In [10]:
req_opt.headers['allow']

'GET, HEAD, OPTIONS'

In [14]:
# 200: 성공, 300: 리디렉션, 400: 클라측 오류, 500: 서버측 오류
req_opt.status_code, type(req_opt.status_code)

(200, int)

In [19]:
req = requests.get('http://httpbin.org/get?myname="seung min"')
req.json()

{'args': {'myname': '"seung min"'},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate, br',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.26.0',
  'X-Amzn-Trace-Id': 'Root=1-61990aec-088ba4f1718c506504246378'},
 'origin': '210.179.33.205',
 'url': 'http://httpbin.org/get?myname="seung min"'}

# XML

In [23]:
input = '''
<students>
    <student x="1">
        <id>001</id>
        <name>Kim</name>
    </student>
    <student x="2">
        <id>002</id>
        <name>Lee</name>
    </student>
</students>
'''

In [24]:
import lxml.etree

문자열에서 읽는 경우

In [32]:
import os
from io import StringIO
# 문자열에서 읽는 경우
root_str = lxml.etree.fromstring(input)
# 파일에서 읽는 경우
tree_file = lxml.etree.parse(os.path.join('data', 'student.xml'))
# 문자열을 파일처럼 읽는 경우
tree_str = lxml.etree.parse(StringIO(input))

fromstring은 root element를 반환하고, parse는 tree를 반환한다.

In [33]:
type(root_str), type(tree_file), type(tree_str)

(lxml.etree._Element, lxml.etree._ElementTree, lxml.etree._ElementTree)

In [34]:
root_file = tree_file.getroot()

In [39]:
print('root.tag = {} ({})'.format(root_str.tag, type(root_str.tag)))

root.tag = students (<class 'str'>)


In [40]:
print('root.attrib = {} ({})'.format(root_str.attrib, type(root_str.attrib)))

root.attrib = {} (<class 'lxml.etree._Attrib'>)


In [41]:
for e in root_str:
    print(e.tag)

student
student


In [43]:
root_str.getchildren(), root_str.getchildren()[0].attrib

([<Element student at 0x7fd61880cec0>, <Element student at 0x7fd618845640>],
 {'x': '1'})

In [50]:
for node in root_str.getiterator():
    print(node.tag, node.attrib, node.text)

students {} 
    
student {'x': '1'} 
        
id {} 001
name {} Kim
student {'x': '2'} 
        
id {} 002
name {} Lee


## 검색 ##

In [51]:
root = root_str

find : 가장 앞에 위치하는 element 만 가져온다.

In [56]:
std = root.find('student')
print(type(std))
for node in std:
    print(node.text)

<class 'lxml.etree._Element'>
001
Kim


findall : 조건에 일치하는 모든 노드를 가져온다.

In [59]:
stds = root.findall('student')
print(type(stds), stds)
for node in stds:
    for item in node:
        print(item.text)

<class 'list'> [<Element student at 0x7fd61880cec0>, <Element student at 0x7fd618845640>]
001
Kim
002
Lee


## XPath ##
- /: 루트 바로 아래만 검색
- //: 어디에 있던 모두 검색
- @: 속성을 검색

In [60]:
root.xpath('//@x')

['1', '2']

In [66]:
# text 뒤에 괄호는 왜 붙는가?
root.xpath('//*[@x="1"]/id/text()')

['001']

## CSSSelect
css를 통해 검색할 수 있게 해준다.  
'.'으로 클래스, '#'으로 아이디 검색을 하게 해준다.

In [68]:
from lxml.cssselect import CSSSelector
sel = CSSSelector('student')
print(type(sel), sel.css, sel.path)

<class 'lxml.cssselect.CSSSelector'> student descendant-or-self::student


In [83]:
# student태그를 모두 검색해서 x출력하기
sel = CSSSelector('student')
nodes = sel(root)
print('type: {}\nlength: {}\nelements: {}'.format(type(nodes), len(nodes), nodes))
for e in nodes:
    print('x:',e.get('x'))

type: <class 'list'>
length: 2
elements: [<Element student at 0x7fd61880cec0>, <Element student at 0x7fd618845640>]
x: 1
x: 2


In [82]:
sel = CSSSelector('id')
nodes = sel(root)
for e in nodes:
    print('id:', e.text)

id: 001
id: 002


# JSON

In [84]:
input = '''[
    {"id" : "001", "x" : "2", "name" : "Chuck"},
    {"id" : "002", "x" : "7", "name" : "Brent"}
]'''

In [87]:
import json
info = json.loads(input)
print('len:', len(info), info[0])

len: 2 {'id': '001', 'x': '2', 'name': 'Chuck'}


### json.dumps
> 주어진 리스트, 딕셔너리를 json형식의 문자열로 변환해준다.
- 튜플을 리스트로, None을 null로 바꿔준다.
- 기본적 문법은 javascript로 되어있다.

In [88]:
dumped = json.dumps(
    ['foo', {'bar': ('baz', None, 1.0, 2)}]
)
type(dumped), dumped

(str, '["foo", {"bar": ["baz", null, 1.0, 2]}]')